# Convex Optimization - Homework 3

<font color='green'>

<i>Report plots, comments and theoretical results in a pdf file. Send your code together with the requested functions and a main script reproducing all your experiments. You can use Matlab, Python or Julia.</i>

Given $x_1,...,x_n \in \mathbb{R}^d$ data vectors and $y_1,...,y_n \in \mathbb{R}$ observations, we are searching for regression parameters $w \in \mathbb{R}^d$ which fit data inputs to observations $y$ by minimizing their squared difference. In a high dimensional setting (when $n \ll d$) a $\ell_1$ norm penalty is
often used on the regression coefficients $w$ in order to enforce sparsity of the solution (so that $w$ will only have a few non-zeros entries). Such penalization has well known statistical properties, and makes the model both more interpretable, and faster at test time. 

From an optimization point of view we want to solve the following problem called LASSO (which stands for Least Absolute Shrinkage Operator and Selection Operator)

\begin{equation}
\tag{LASSO}
\begin{array}{ll}
\text{minimize} & \frac{1}{2} \left\lVert Xw - y \right\rVert^2_2 + \lambda \left\lVert w \right\rVert_1
\end{array}
\end{equation}

in the variable $w \in \mathbb{R}^d$, where $X = (x^T_1, \ldots, x^T_n) \in \mathbb{R}^{n\times d},\, y = (y_1, \ldots, y_n) \in \mathbb{R}^n$ and $\lambda > 0$ is a regularization parameter.
</font>

<font color='navyblue'>
1. Derive the dual problem of LASSO and format it as a general Quadratic Problem as follows


\begin{equation}
\tag{QP}
\begin{array}{ll}
\text{minimize} & v^TQv + p^Tv \\
\text{subject to} & Av \preceq b
\end{array}
\end{equation}

in variable $v\in \mathbb{R}^n$, where $Q \succeq 0$.
</font>

We can repose the LASSO problem as the following problem

\begin{equation}
\tag{LASSO'}
\begin{array}{ll}
\displaystyle \min_{w \in \mathbb{R}^d,\, z \in \mathbb{R}^n} & \frac{1}{2} \left\lVert z \right\rVert^2_2 + \lambda \left\lVert w \right\rVert_1 \\
\text{subject to} & z = Xw - y 
\end{array}
\end{equation}

The associated Lagragian is

\begin{align*}
\mathcal{L}(w, z, v) &= \frac{1}{2} z^T z + \lambda \left\lVert w \right\rVert_1 + v^T (y-Xw+z)
\end{align*}

We can then calculate the dual function

\begin{align*}
g(v) &= \inf_{w, z} \frac{1}{2} z^T z + \lambda \left\lVert w \right\rVert_1 + v^T (y-Xw+z) \\
&= v^T y + \inf_z \{ \frac{1}{2} z^T z + v^T z\} + \lambda \inf_w \{\left\lVert w \right\rVert_1 - \frac{1}{\lambda} v^T X w\} \\
&= \left\{ \begin{array}{ll} v^T y - \frac{1}{2} v^T v & \text{if } \left\lVert X^T v \right\lVert_{\infty} \le \lambda \\
- \infty & \text{otherwise} \end{array}\right.
\end{align*}

We can obtain the dual problem 

\begin{equation}
\tag{LASSO*}
\begin{array}{ll}
\text{maximize} & v^T y - \frac{1}{2} v^T v \\
\text{subject to} & \left\lVert X^T v \right\lVert_{\infty} \le \lambda \\
\end{array}
\end{equation}

which can be simplified as follows

\begin{equation}
\tag{LASSO*}
\begin{array}{ll}
\displaystyle \min_{v \in \mathbb{R}^n} & \frac{1}{2} v^T v  - y^T v \\
\text{subject to} & - X^T v \le \lambda \cdot \mathbb{1}_d \\
& X^T v \le \lambda \cdot \mathbb{1}_d
\end{array}
\end{equation}

Finally, we can rewrite it as 

\begin{equation}
\tag{QP}
\begin{array}{ll}
\text{minimize} & v^TQv + p^Tv \\
\text{subject to} & Av \preceq b
\end{array}
\end{equation}

with

- $Q = \frac{1}{2} I_n \in \mathbb{R}^{n \times n}$, we have $Q \succeq 0$.
- $p = -y \in \mathbb{R}^{n}$
- $
A = \left(\begin{array}{c}
X^T \\
\hline -X^T
\end{array}\right)
\in \mathbb{R}^{2d \times n}$
- $b = \lambda \cdot \mathbb{1}_{2d} \in \mathbb{R}^{2d}$

For the next question, we pose $m=2d$.


<font color='navyblue'>
1. Impliment the barrier method to solve QP.

- Write a function `v_seq = centering_step(Q, p, A, b, t, v0, eps)` which impliments the Newton method to solve the centering step given the inputs $(Q, p, A, b)$, the barrier method parameter $t$ (see lectures), initial variable $v_0$ and a target precision $\epsilon$. The function outputs the sequence of variables iterates $(v_i)_{i=1, \ldots, n_{\epsilon}}$, where $n_{\epsilon}$ is the number of iterations to obtain the $\epsilon$ precision. Use a backtracking line search with appropriate parameters.
- Write a function `v_seq = barr_method(Q, p, A, b, v0, eps)` which implements the barrier method to solve QP using precedent function given the data inputs $(Q, p, A, b)$, a feasible point $v_0$, a precision criterion $\epsilon$. The function ouptuts the sequence of variables iterates $(v_i)_{i=1, \ldots, n_{\epsilon}}$, where $n_{\epsilon}$ is the number of iterations to obtain the $\epsilon$ precision.
- Test your function on randomly generated matrices $X$ and observations $y$ withz $\lambda = 10$. Plot precision criterion and gap $f(v_t) - f^*$ in semilog scale (using the best value found for $f$ as a surrogate for $f^*$). Repeat for different values of the barrier method parameter $\mu = 2, 15, 50, 100, \ldots$ and check the impact on $w$. What would be an appropriate choice for $\mu$ ?
</font>

## Implémentation avec CVXPY

In [1]:
import cvxpy as cp
import numpy as np

In [7]:
# Problem data.
d = 30
n = 20
ld = 10

np.random.seed(1)
X = np.random.randn(n, d)
y = np.random.randn(n)

Q = 0.5 * np.eye(n)
p = -y
A = np.vstack((X.transpose(), -X.transpose()))
b = ld * np.ones(2*d)

In [9]:
X.shape, A.shape, b.shape

((20, 30), (60, 20), (60,))

In [18]:
x = np.random.randn(n)
x.T @ Q @ x + p.T @ x

15.8097915781266

In [19]:
# Construct the problem.
w = cp.Variable(d)
objective = cp.Minimize(0.5 * cp.sum_squares(X @ w - y) + ld * cp.norm1(w))
constraints = []
prob = cp.Problem(objective, constraints)

In [23]:
# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(w.value)

[-1.49638832e-16  2.41755363e-16  1.13010118e-17  9.80215835e-18
 -3.20979664e-17  1.51703003e-17 -1.88212106e-16  9.29312805e-18
  3.09124339e-17 -1.84703512e-16  1.83428588e-16 -8.22773022e-17
  4.07999310e-16  1.67822357e-16  4.96226074e-17 -2.01872854e-16
 -9.13446915e-17 -2.91577062e-16 -5.69568658e-02 -1.97740536e-17
  2.15999264e-16  1.30042830e-16  2.75535912e-16  2.52588184e-16
  1.29454912e-16  2.22406743e-16  7.53568342e-17 -2.34962309e-16
  1.83025671e-17  7.87841190e-18]


## Implementation à la main

In [4]:
import numpy as np
import scipy as sc

from typing import Callable, Tuple

f_type = Callable[[np.array], float]


In [30]:
def backtracking_line_search(
    x: np.array, f: f_type, grad_f: f_type, delta_x: np.array, alpha: float, beta: float
) -> float:
    assert alpha > 0 and alpha < 1 / 2, "Alpha must be between 0 and 0.5"
    assert beta > 0 and beta < 1, "Beta must be between 0 and 1"

    t = 1
    while not (f(x + t * delta_x) < f(x) + alpha * t * grad_f(x) @ delta_x):
        t *= beta

    return t


def newton_step(x: np.array, grad_f: f_type, hess_f: f_type) -> np.array:
    step = -np.linalg.inv(hess_f(x)) @ grad_f(x)
    return step


def newton_decrement(x: np.array, grad_f: f_type, hess_f: f_type) -> float:
    decrement = np.sqrt(grad_f(x).transpose() @ np.linalg.inv(hess_f(x)) @ grad_f(x))
    return decrement


def newton_method(
    x0: np.array,
    f: f_type,
    grad_f: f_type,
    hess_f: f_type,
    epsilon: float,
    alpha: float = 0.1,
    beta: float = 0.5,
    verbose=True
):

    x = x0
    decrement = newton_decrement(x, grad_f, hess_f)

    xs = [x0]
    decrements = [decrement]
    steps = []
    ts = []

    while decrement**2 / 2 > epsilon:
        if verbose: print(f"Stopping value: {decrement**2:.2f} < {epsilon}")
        decrement = newton_decrement(x, grad_f, hess_f)
        step = newton_step(x, grad_f, hess_f)
        t = backtracking_line_search(x, f, grad_f, step, alpha, beta)
        x += t * step

        decrements.append(decrement)
        steps.append(step)
        ts.append(t)
        xs.append(x)

    return x, f(x), {"xs": xs, "decrements": decrements, "steps": steps, "ts": ts}


We can write

$$
A = \left(\begin{array}{ccc}
& a_1^T & \\
\hline
& \vdots & \\
\hline
& a_m^T &
\end{array}\right)
$$
with $a_i \in \mathbb{R}^n$.

The barrier problem can be written as

\begin{equation}
\tag{Barrier}

\begin{array}{ll}
\displaystyle \min_{v \in \mathbb{R}^n} & t (v^T Q v + p^T v) + \phi(v) \\
\text{subject to} & \forall i \in [1, \cdots, m], \; a_i^T v - b_i \le 0
\end{array}

\end{equation}

We pose $\forall i \in [1, \cdots, m], \; f_i(v) =  a_i^T v - b_i \in \mathbb{R}$. With this notation, $\phi(v) = - \displaystyle \sum_{i=1}^m \log(-f_i(v))$.

From there, we can look at

\begin{align*}
\nabla \phi(v) &= \sum_{i=1}^m \frac{1}{- f_i(v)} \nabla f_i(v) \\
&= \sum_{i=1}^m \frac{1}{b_i - a_i^T v} a_i
\end{align*}

and

\begin{align*}
\nabla^2 \phi(v) &= \sum_{i=1}^m \frac{1}{f_i(v)^2} \nabla f_i(v) \nabla f_i(v)^T \\
&= \sum_{i=1}^m \frac{1}{(b_i - a_i^T v)^2} a_i a_i^T
\end{align*}

since $\nabla^2 f_i(v) = 0$.






In [31]:
def f(v, Q, p):
    """The original objective function f_0."""
    return v.transpose() @ Q @ v + p.transpose() @ v


def grad_f(v, Q, p):
    """The gradient of f."""
    return (Q + Q.transpose()) @ v + p


def hess_f(v, Q, p):
    """The hessian of f."""
    return Q


def g(v, A, b, i):
    """The inequality constraints f_i."""
    aT = A[i, :]
    return aT @ v - b[i]


def grad_g(v, A, b, i):
    """The gradient of the nequality constraints f_i."""
    aT = A[i, :]
    return aT.transpose()


def phi(v, A, b):
    """The log barrier function phi."""
    m = len(b)
    return -np.sum([np.log(-g(v, A, b, i)) for i in range(m)])


def grad_phi(v, A, b):
    """The gradient of phi."""
    m = len(b)
    l = [1 / (-g(v, A, b, i)) * grad_g(v, A, b, i) for i in range(m)]
    return np.sum(l)


def hess_phi(v, A, b):
    """The hessian of phi."""
    m = len(b)
    l = [
        1 / (g(v, A, b, i) ** 2) * grad_g(v, A, b, i) @ grad_g(v, A, b, i).transpose()
        for i in range(m)
    ]
    return np.sum(l)


In [32]:
def centering_step(Q, p, A, b, t, v0, eps, alpha=0.1, beta=0.5, verbose=True):
    x_opt, f_opt, iterates = newton_method(
        x0=v0,
        f=lambda v: t * f(v, Q, p) + phi(v, A, b),
        grad_f=lambda v: t * grad_f(v, Q, p) + grad_phi(v, A, b),
        hess_f=lambda v: t * hess_f(v, Q, p) + hess_phi(v, A, b),
        epsilon=eps,
        alpha=alpha,
        beta=beta,
        verbose=verbose
    )

    return iterates["xs"]


In [46]:
def barr_method(Q, p, A, b, v0, eps, t0=1, mu=15, alpha=0.1, beta=0.5, verbose=True):
    t = t0
    m = len(b)
    v = v0
    vs = [v]

    while m / t >= eps:
        if verbose: print("t =", t)
        v = centering_step(Q, p, A, b, t, v, eps, alpha, beta, verbose=verbose)[-1]
        t *= mu

        vs.append(v)

    return vs


In [47]:
lamb = 10
n = 20
d = 50
X = np.random.randn(n, d)
y = np.random.rand(n)

In [48]:
Q = 0.5 * np.eye(n)
p = -y
A = np.vstack((X.transpose(), -X.transpose()))
b = lamb * np.ones((2*d))
v0 = np.zeros(n)

In [50]:
steps = barr_method(Q, p, A, b, v0, eps=0.5, t0=10, mu=15, alpha=0.1, beta=0.5)

t = 10
Stopping value: 11.15 < 0.5
Stopping value: 11.15 < 0.5
t = 150
Stopping value: 171.46 < 0.5
Stopping value: 171.46 < 0.5
Stopping value: 74.84 < 0.5
Stopping value: 32.03 < 0.5
Stopping value: 13.72 < 0.5
Stopping value: 5.96 < 0.5
Stopping value: 2.64 < 0.5
Stopping value: 1.20 < 0.5


In [41]:
fs = [f(v, Q, p) for v in steps]

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [51]:
steps

[array([ 0.11663611,  0.22020262,  0.65808571,  0.35437959, -0.05070135,
         0.43086586,  0.60822425,  0.51965467,  0.07386646, -0.05863175,
         0.61821539,  0.05333455,  0.90812095,  0.86598096,  0.18265423,
        -0.0536297 ,  0.04928338,  0.48473493,  0.60911364,  0.84605453]),
 array([ 0.11663611,  0.22020262,  0.65808571,  0.35437959, -0.05070135,
         0.43086586,  0.60822425,  0.51965467,  0.07386646, -0.05863175,
         0.61821539,  0.05333455,  0.90812095,  0.86598096,  0.18265423,
        -0.0536297 ,  0.04928338,  0.48473493,  0.60911364,  0.84605453]),
 array([ 0.11663611,  0.22020262,  0.65808571,  0.35437959, -0.05070135,
         0.43086586,  0.60822425,  0.51965467,  0.07386646, -0.05863175,
         0.61821539,  0.05333455,  0.90812095,  0.86598096,  0.18265423,
        -0.0536297 ,  0.04928338,  0.48473493,  0.60911364,  0.84605453])]